In [1]:
!ls *.csv

json_tuple_example.csv	marks.csv  marks_without_header.csv  weatherHistory.csv


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Json Tuple").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
! cat json_tuple_example.csv

PartitionDate,Status,request
2020-06-30,Internal Error,"{""Response"":{""MessageId"":15432}}"
2020-06-30,Success,"{""Response"":{""MessageId"":15432,""Lattitude"":""-176.2989"",""Longitude"":""7.3614""}}"


In [5]:
df_json_csv = spark.read.csv("json_tuple_example.csv",multiLine=True,escape="\"",header=True)

In [6]:
df_json_csv.show(truncate=0)

+-------------+--------------+-----------------------------------------------------------------------------+
|PartitionDate|Status        |request                                                                      |
+-------------+--------------+-----------------------------------------------------------------------------+
|2020-06-30   |Internal Error|{"Response":{"MessageId":15432}}                                             |
|2020-06-30   |Success       |{"Response":{"MessageId":15432,"Lattitude":"-176.2989","Longitude":"7.3614"}}|
+-------------+--------------+-----------------------------------------------------------------------------+



### Method 1 : Using JSON Tuple

In [20]:
from pyspark.sql.functions import col,json_tuple,to_json,from_json

In [21]:
df_json_level_1 = df_json_csv.select("*",json_tuple("request","Response")).drop("request")
df_json_level_1.show(truncate=0)

+-------------+--------------+----------------------------------------------------------------+
|PartitionDate|Status        |c0                                                              |
+-------------+--------------+----------------------------------------------------------------+
|2020-06-30   |Internal Error|{"MessageId":15432}                                             |
|2020-06-30   |Success       |{"MessageId":15432,"Lattitude":"-176.2989","Longitude":"7.3614"}|
+-------------+--------------+----------------------------------------------------------------+



In [23]:
df_json_level_2 = df_json_level_1.select("*",json_tuple("c0","MessageId","Lattitude","Longitude").alias("MessageID","Lattitude","Longitude")).drop("c0")

In [24]:
df_json_level_2.show()

+-------------+--------------+---------+---------+---------+
|PartitionDate|        Status|MessageID|Lattitude|Longitude|
+-------------+--------------+---------+---------+---------+
|   2020-06-30|Internal Error|    15432|     null|     null|
|   2020-06-30|       Success|    15432|-176.2989|   7.3614|
+-------------+--------------+---------+---------+---------+



## Metjod 2 : from_json  :; Prefereable

In [25]:
df_json_csv.select(col('request').alias("jsoncol")).rdd.map(lambda x:x.jsoncol).collect()

['{"Response":{"MessageId":15432}}',
 '{"Response":{"MessageId":15432,"Lattitude":"-176.2989","Longitude":"7.3614"}}']